In [ ]:
# (C) Copyright IBM Corp. 2019, 2020, 2021, 2022.

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#           http://www.apache.org/licenses/LICENSE-2.0

#     Unless required by applicable law or agreed to in writing, software
#     distributed under the License is distributed on an "AS IS" BASIS,
#     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#     See the License for the specific language governing permissions and
#     limitations under the License.


In [4]:
import random
import os
import optuna
from argparse import ArgumentParser
import os 

os.environ['engine'] = 'pytorch'

import numpy as np
import matplotlib.pyplot as plt
from simulai.utilities.oscillator_solver import oscillator_solver_forcing
from simulai.workflows import StepwiseExtrapolation
from simulai.regression import EchoStateNetwork
from simulai.templates import HyperTrainTemplate
from simulai.workflows import ParamHyperOpt
from simulai.metrics import L2Norm

Hyperparameter search for  ESN+ModelPool using Optuna.

In [5]:

class HyperModelPoolESN(HyperTrainTemplate):

    def __init__(self, trial_config: dict=None, set_type='hard', other_params: dict=None):

        self.model = None

        self.others_params = other_params

        required_keys = ['sub_model_number_of_inputs', 'global_matrix_constructor_str',
                         'n_workers', 'solver', 'initial_state', 'path_to_save', 'tag', 'id']

        self.sub_model_number_of_inputs = None
        self.global_matrix_constructor_str = None
        self.n_workers = None
        self.solver = None
        self.initial_state = None
        self.n_steps = None
        self.path_to_save = None
        self.tag = 'model_'
        self.id = None

        for key in required_keys:
            assert key in trial_config.keys(), f"The required parameter {key} is not in others_params."

        for key, value in trial_config.items():
            setattr(self, key, value)

        super().__init__(trial_config=trial_config, set_type=set_type)

        self.model_id = self.tag + str(self.id)

    def _set_model(self):

        rc_config = {
                     'reservoir_dim': self.trial_config['reservoir_dim'],
                     'sparsity_level': self.trial_config['sparsity_level'] * self.trial_config['reservoir_dim'],
                     'radius': self.trial_config['radius'],
                     'sigma': self.trial_config['sigma'],
                     'beta': 10**self.trial_config['beta_exp']
                    }

        extra_params = {
                        'number_of_inputs': self.sub_model_number_of_inputs,
                        'global_matrix_constructor_str': self.global_matrix_constructor_str,
                        'solver': self.solver,
                        'n_workers': self.n_workers
                       }

        rc_config.update(extra_params)

        self.model = EchoStateNetwork(**rc_config)

    def fit(self, input_train_data=None, target_train_data=None):

        self.model.fit(input_data=input_train_data, target_data=target_train_data)

        self.model.save_model(save_path=self.path_to_save, model_name=self.model_id)
        

Custommization wrapper class.

In [6]:
class ObjectiveWrapper:

        def __init__(self, test_data=None, forcings_input=None, initial_state=None, n_steps=None):

            self.test_data = test_data
            self.forcings_input = forcings_input
            self.initial_state = initial_state
            self.n_steps = n_steps

        def __call__(self, trainer_instance=None, objective_function=None):

            return objective_function(model=trainer_instance, initial_state=self.initial_state,
                                      test_data=self.test_data,
                                      forcings_input=self.forcings_input,
                                      n_steps=self.n_steps)


Objective function.

In [7]:
def objective(model=None, initial_state=None, test_data=None, forcings_input=None, n_steps=None):

    extrapolator = StepwiseExtrapolation(model=model.model, keys=["ESN_0"])

    l2_norm = L2Norm()

    estimated_data = extrapolator.predict(initial_state=initial_state,
                                          auxiliary_data=forcings_input,
                                          horizon=n_steps)

    error = 100 * l2_norm(data=estimated_data, reference_data=test_data, relative_norm=True)

    return error

Problem parameters.

In [9]:
n_steps = 1000
A = 1
T = 50
dt = T / n_steps

Generating a random forcing array (or restoring it).

In [10]:
if not os.path.isfile('forcings.npy'):
    forcings = A * np.random.rand(n_steps, 2)
    np.save('forcings.npy', forcings)
else:
    forcings = np.load('forcings.npy')

Numbers of field and forcing variables.

In [11]:
n_field = 2  
n_forcing = 2 
sub_model_number_of_inputs = n_field + n_forcing

Datasets fractions.

In [12]:
train_fraction = 0.6
validation_fraction = 0.3
test_fraction = 0.1

n_train= int(train_fraction * n_steps)  # size of time steps
n_validation = int(validation_fraction * n_steps)
n_test = int(test_fraction * n_steps)

Number of multiprocess workers and trials.

In [13]:
n_workers = 8
n_trials = 5

Generating datasets.

In [15]:
initial_state = np.array([2, 0])[None, :]
oscillator_data, _ = oscillator_solver_forcing(T, dt, initial_state, forcing=forcings)

Iteration 999

Preparing datasets to be used by the Echo-State networks.

In [17]:
field_data = oscillator_data

train_data = field_data[:n_train, :]
validation_data = field_data[n_train:n_train+n_validation, :]
test_data = field_data[n_train+n_validation:, :]

input_data = train_data[:-1, :]
target_data = train_data[1:, :]

forcings_train_data = forcings[:n_train, :][:-1]

forcings_validation_data = forcings[n_train:n_train+n_validation, :]

forcings_test_data = forcings[n_train+n_validation:, :]

initial_state_validation = train_data[-1:, :]
initial_state_test = validation_data[-1:, :]

Parameters for the hyper-search.

In [18]:
 params_intervals = {
        'reservoir_dim': (1000, 2000),
        'sparsity_level': (0.05, 0.1),
        'radius': (0.3, 0.7),
        'sigma': (0.3, 0.7),
        'beta_exp': (-5, -2)
    }

params_suggestions = {
    'reservoir_dim': 'int',
    'sparsity_level': 'float',
    'radius': 'float',
    'sigma': 'float',
    'beta_exp': 'int'
}

Others parameters, fixed by default.

In [20]:
path_to_save = '.'
others_params = {'number_of_inputs': sub_model_number_of_inputs,
                 'sub_model_number_of_inputs': sub_model_number_of_inputs,
                 'global_matrix_constructor_str': 'multiprocessing',
                 'solver': 'linear_system',
                 'n_workers': n_workers,
                 'initial_state': initial_state_validation,
                 'n_steps': n_validation,
                 'tag': 'model_',
                 'path_to_save': path_to_save}

Wrapper for the objective function in order to enable communication with the hyper-search engine.

In [21]:
objective_wrapper = ObjectiveWrapper(test_data=validation_data,
                                     forcings_input=forcings_validation_data,
                                     initial_state=initial_state_validation,
                                     n_steps=n_validation)

Instantiating the hyper-search engine.

In [22]:
hyper_search = ParamHyperOpt(params_intervals=params_intervals,
                             params_suggestions=params_suggestions,
                             name='oscillator_search',
                             direction='minimize',
                             trainer_template=HyperModelPoolESN,
                             objective_wrapper=objective_wrapper,
                             objective_function=objective,
                             others_params=others_params,
                             refresh=True)

Setting up datasets.

In [23]:
hyper_search.set_data(input_train_data=input_data,
                          target_train_data=target_data,
                          auxiliary_train_data=forcings_train_data,
                          input_validation_data=validation_data,
                          target_validation_data=validation_data,
                          auxiliary_validation_data=forcings_validation_data,
                          input_test_data=test_data,
                          target_test_data=test_data,
                          auxiliary_test_data = forcings_test_data)

Executing hyper-search.

In [24]:
hyper_search.optimize(n_trials=n_trials)

Creating an instance from <class '__main__.HyperModelPoolESN'>
Initializing ESN matrices ...
Evaluating the hidden state ...
 state 598
Applying transformation ...
Constructing W_out ...
Using multiprocessing engine.Using multiprocessing engine.Using multiprocessing engine.


Using multiprocessing engine.
Using multiprocessing engine.Using multiprocessing engine.
Using multiprocessing engine.

Using multiprocessing engine.
Solving the linear system using the most proper algorithm.
Fitting concluded.


[I 2021-12-14 13:32:19,301] Finished trial#0 with value: 21.175802363224754 with parameters: {'reservoir_dim': 1706, 'sparsity_level': 0.08060348200971132, 'radius': 0.5864432482632342, 'sigma': 0.3916357208788437, 'beta_exp': -4}. Best is trial#0 with value: 21.175802363224754.


Creating an instance from <class '__main__.HyperModelPoolESN'>
Initializing ESN matrices ...
Evaluating the hidden state ...
 state 598
Applying transformation ...
Constructing W_out ...
Using multiprocessing engine.Using multiprocessing engine.
Using multiprocessing engine.Using multiprocessing engine.


Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.
Solving the linear system using the most proper algorithm.
Fitting concluded.


[I 2021-12-14 13:32:32,190] Finished trial#1 with value: 21.239974805492263 with parameters: {'reservoir_dim': 1396, 'sparsity_level': 0.08844244499075869, 'radius': 0.5732247740821035, 'sigma': 0.4640142031323618, 'beta_exp': -4}. Best is trial#0 with value: 21.175802363224754.


Removing ./model_1.
Creating an instance from <class '__main__.HyperModelPoolESN'>
Initializing ESN matrices ...
Evaluating the hidden state ...
 state 598
Applying transformation ...
Constructing W_out ...
Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.Using multiprocessing engine.

Using multiprocessing engine.Using multiprocessing engine.

Using multiprocessing engine.
Solving the linear system using the most proper algorithm.
Fitting concluded.


[I 2021-12-14 13:32:50,470] Finished trial#2 with value: 22.67665135741441 with parameters: {'reservoir_dim': 1728, 'sparsity_level': 0.07350990984026284, 'radius': 0.3700811907693127, 'sigma': 0.4860410208165875, 'beta_exp': -2}. Best is trial#0 with value: 21.175802363224754.


Removing ./model_2.
Creating an instance from <class '__main__.HyperModelPoolESN'>
Initializing ESN matrices ...
Evaluating the hidden state ...
 state 598
Applying transformation ...
Constructing W_out ...
Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.Using multiprocessing engine.

Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.
Solving the linear system using the most proper algorithm.
Fitting concluded.


[I 2021-12-14 13:33:12,008] Finished trial#3 with value: 23.122198919706157 with parameters: {'reservoir_dim': 1952, 'sparsity_level': 0.051454621461581036, 'radius': 0.6072483582768003, 'sigma': 0.4090629560745167, 'beta_exp': -2}. Best is trial#0 with value: 21.175802363224754.


Removing ./model_3.
Creating an instance from <class '__main__.HyperModelPoolESN'>
Initializing ESN matrices ...
Evaluating the hidden state ...
 state 598
Applying transformation ...
Constructing W_out ...
Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.
Using multiprocessing engine.Using multiprocessing engine.
Using multiprocessing engine.

Using multiprocessing engine.
Solving the linear system using the most proper algorithm.
Fitting concluded.


[I 2021-12-14 13:33:35,259] Finished trial#4 with value: 22.641057589052078 with parameters: {'reservoir_dim': 1954, 'sparsity_level': 0.052362531261062946, 'radius': 0.383784679010151, 'sigma': 0.36867382710790075, 'beta_exp': -3}. Best is trial#0 with value: 21.175802363224754.
